In [1]:
# CE 295 - Energy Systems and Control
# Final Project: Model Predictive Control and the Optimal Power Flow Problem in the IEEE 39-bus Test Feededr
# Authors (alphabetical): Carla Becker, Hannah Davalos, Jean-Luc Lupien, John Schafer, Keyi Yang
# Adapted from code provided by Prof. Daniel B. Arnold

import numpy as np
import matplotlib.pyplot as plt
from cvxpy import *
import pandas as pd

In [2]:
# Import data for IEEE 39-bus Test Feeder

# Define transformer resistance and reactance (not provided by test feeder)
xformer_r = 0.0001
xformer_x = 0.0015

# Adjacency matrix; assumes power can flow in two directions on all lines
A_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='A matrix')
A_df = A_df.iloc[:,1:] # remove the column labels
A    = A_df.values     # convert to a numpy array

# Resistance matrix
r_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='r matrix')
r_df = r_df.iloc[:,1:] # remove the column labels
r_df.replace('??', xformer_r, inplace=True)
r    = r_df.values     # convert to a numpy array

# Reactance matrix
x_df = pd.read_excel('IEEE 39 Test Bus Data/IEEE_39_bus_data.xlsx', sheet_name='x matrix')
x_df = x_df.iloc[:,1:] # remove the column labels
x_df.replace('??', xformer_x, inplace=True)
x    = x_df.values     # convert to a numpy array

In [3]:
## 39 Node IEEE Test Feeder Parameters

### Node (aka Bus) Data
# l_j^P: Active power consumption [MW]
cons_real_power = np.array([[0,0.2,0,0.4,0.17,0.23,1.155,0,0.17,0.843,0, 0.17,0.128,0,0.2,0,0.4,0.17,0.23,1.155,0,0.17,0.843,0, 0.17,0.128,0,0.2,0,0.4,0.17,0.23,1.155,0,0.17,0.843,0, 0.17,0.128]])

# l_j^Q: Reactive power consumption [MVAr]
cons_reactive_power = np.array([[0,0.116,0,0.29,0.125,0.132,0.66,0,0.151,0.462,0,0.08,0.086,0,0.116,0,0.29,0.125,0.132,0.66,0,0.151,0.462,0,0.08,0.086,0,0.116,0,0.29,0.125,0.132,0.66,0,0.151,0.462,0,0.08,0.086]])

# l_j^S: Apparent power consumption [MVA]
cons_apparent_power = np.sqrt(cons_real_power**2 + cons_reactive_power**2)

# s_j,max: Maximal generating power [MW]
max_apparent_power = np.array([5,0,0,3,0,0,0,0,0,3,0,0,0,5,0,0,3,0,0,0,0,0,3,0,0,0,5,0,0,3,0,0,0,0,0,3,0,0,0])
max_apparent_power = max_apparent_power.reshape(1, 39)

# c_j: Marginal generation cost [USD/MW]
c = np.array([[100],
              [0],
              [0],
              [150],
              [0],
              [0],
              [0],
              [0],
              [0],
              [50],
              [0],
              [0],
              [0],
              [100],
              [0],
              [0],
              [150],
              [0],
              [0],
              [0],
              [0],
              [0],
              [50],
              [0],
              [0],
              [0],
              [100],
              [0],
              [0],
              [150],
              [0],
              [0],
              [0],
              [0],
              [0],
              [50],
              [0],
              [0],
              [0]])

# V_min, V_max: Minimum and maximum nodal voltages [V]
min_voltage = 0.95
max_voltage = 1.05

# v_min = 0.98
# v_max = 1.02

# I_max_ij: Maximal line current [p.u.]
I_max = A * 0.01

### Set Data
# List of node indices
j_idx = np.arange(13)

# \rho(j): Parent node of node j
rho = np.array([0,0,1,2,1,4,1,6,6,8,6,10,10,0,0,1,2,1,4,1,6,6,8,6,10,10,0,0,1,2,1,4,1,6,6,8,6,10,10])

# Renewable nodes
renew_nodes = np.array([30, 32, 33, 34, 35, 36, 37, 38])

In [4]:
# PLACEHOLDER FOR CONSUMPTION PLOTTING

In [68]:
# Function for convex optimization in 1 time step
def cvx_optim(cons_real_power, cons_reactive_power, max_apparent_power, min_voltage, max_voltage):

    # TODO Carla, change inputs to be matrices, not vectors (cover all time)

    horizon   = cons_real_power.shape[0]
    num_nodes = cons_real_power.shape[1]
    
    # Define optimization vars
    gen_real_power       = Variable((num_nodes, horizon))
    gen_reactive_power   = Variable((num_nodes, horizon))
    gen_apparent_power   = Variable((num_nodes, horizon))
    line_real_power      = [Variable((num_nodes, num_nodes)) for i in range(horizon)]
    line_reactive_power  = [Variable((num_nodes, num_nodes)) for i in range(horizon)]
    line_voltages        = Variable((num_nodes, horizon))
    line_complex_current = [Variable((num_nodes, num_nodes)) for i in range(horizon)]
    sigma_A              = Variable(len(renew_nodes))

    #BESS
    BESS_energy= Variable((num_nodes, horizon))
    BESS_chrg_dis= Variable((num_nodes, horizon))

    # Robust optimization decision vars
    y = vstack([sigma_A[0], gen_apparent_power[0]])
    for i in range(1, len(renew_nodes)):
        y = vstack([y, vstack([sigma_A[i], gen_apparent_power[i]])])

    # Robust optimization parameters
    a = np.array([-1.25, 1])
    bar_a = np.tile(a, len(renew_nodes))

    e_array = np.array((0.25, 0))
    E = np.diag(np.tile(e_array, len(renew_nodes)))

    #BESS Parameters
    timestep=.25 # assuming 15min timestep
    eta=.95 #(charging /discharging efficiency)
    energy_min=0 #minimun energy level of BESS, need to define for every node seperately?
    #100 max energy level of BESS, in kWh NEED TO SCALE
    # TODO Carla, set nodes that have battery storage
    energy_max=np.array([0, 0, 0,0, 100, 100, 0, 0, 0, 0, 0]) #placeholder, need actual nodes that have BESS, needs to have 39 nodes
    
    ramp_max= 100 / timestep #1C, can discharge/charge 1 full capacity every hour, assuming timestep is in hours
    initial_energy=np.zeros(num_nodes) #place holder
    #where to I specify which nodes are BESS?

    # BESS initial state
    constraints = [BESS_energy[:, 0] == initial_energy]  # initial_energy needs to be defined

    # Define objective function
    objective =  Minimize(sum(c.T @ gen_apparent_power))

    # Define constraints
    # Apparent Power Limits
    constraints += [gen_apparent_power <= max_apparent_power]

    # Boundary condition for power line flows
    constraints += [line_real_power[0][0]     == 0,
                    line_reactive_power[0][0] == 0]

    # Boundary condition for squared line current
    constraints += [line_complex_current[0][0] == 0]

    # Fix node 0 voltage to be 1 "per unit" (p.u.)
    constraints += [line_voltages[0] == 1]

    # Robust Optimization Constraints
    constraints += [bar_a @ y + norm(E @ y) <= 0]
    constraints += [sigma_A >= 0, sigma_A <= 1]

    # Loop over every time step in the horizon, loop over each nodes in each time step
    for t in range(horizon):
        for jj in [0]:#j_idx:
            
            #BESS static Constraints
            constraints +=[energy_min<=BESS_energy[jj, t], BESS_energy[jj, t] <= energy_max[jj]] #every battery node has energy limits for every time

            if t < horizon - 1:
                # Ramp rate constraints
                constraints += [BESS_chrg_dis[jj, t + 1] - BESS_chrg_dis[jj, t] <= ramp_max,
                                BESS_chrg_dis[jj, t] - BESS_chrg_dis[jj, t + 1] <= ramp_max]

                # BESS energy dynamics
                constraints += [BESS_energy[jj, t + 1] == BESS_energy[jj, t] + timestep * eta * BESS_chrg_dis[jj, t]]

            # Nodal voltage limits, not dependent on t
            constraints += [min_voltage**2 <= line_voltages[jj,t]]
            constraints += [line_voltages[jj,t] <= max_voltage**2]

            # Non-negative power generation
            constraints += [gen_real_power[jj, t] >= 0]
            constraints += [gen_reactive_power[jj, t] >= 0]

            # Parent node, i = rho(j)
            ii = rho[jj]

            # Squared line current limits
            constraints += [line_complex_current[ii][jj] <= (I_max[ii][jj])**2] # I DONT KNOW HOW TO INDEX BY t

            # Line Power Flows
            constraints += [line_real_power[ii][jj]     == (cons_reactive_power[jj, t] - gen_apparent_power[jj,  t]) + r[ii][jj]*line_complex_current[ii][jj] + A[jj]@line_real_power[jj].T]
            constraints += [line_reactive_power[ii][jj]     == (cons_reactive_power[jj, t] - gen_apparent_power[jj,  t]) + x[ii][jj]*line_complex_current[ii][jj] + A[jj]@line_real_power[jj].T]

            # Nodal voltage
            constraints += [line_voltages[jj,t] == line_voltages[ii,t] + ((r[ii][jj])**2 + (x[ii][jj])**2)*line_complex_current[ii][jj] - 2*(r[ii][jj]*line_real_power[ii][jj]+x[ii][jj]*line_reactive_power[ii][jj])]
            
            # Squared current magnitude on lines
            constraints += [line_complex_current[ii][jj] >= quad_over_lin(vstack([line_real_power[ii][jj],line_reactive_power[ii][jj]]),line_voltages[jj,t])]

            # Compute apparent power from active & reactive power
            constraints += [norm(vstack([gen_real_power[jj,t], gen_reactive_power[jj,t]])) <= gen_apparent_power[jj,t]]

    # Define problem and solve
    prob = Problem(objective, constraints)
    prob.solve()


    # Output Results
    print(prob.status)
    print(f"Minimum Generating Cost: {prob.value} USD")

    print(" ")
    print(f"Node 30 [Diesel]: real power = {gen_real_power[30].value} MW | reactive power = {gen_reactive_power[30].value} MVAr | apparent power = {gen_apparent_power[30].value} MVA")
    
    print(" ")
    for node in renew_nodes:
        print(f"Node {node} [Renewable]: real power = {gen_real_power[node].value} MW | reactive power = {gen_reactive_power[node].value} MVAr | apparent power = {gen_apparent_power[node].value} MVA")
    
    print(" ")
    #print(f"Total active real power:     {np.sum(line_real_power)} MW consumed | {np.sum(gen_real_power)} MW generated")
    #print(f"Total active reactive power: {np.sum(line_reactive_power)} MW consumed | {np.sum(gen_reactive_power)} MW generated")
    

In [69]:
# Test csv_optim function
cvx_optim(cons_real_power, cons_reactive_power, max_apparent_power, min_voltage, max_voltage)

unbounded
Minimum Generating Cost: -inf USD
 
Node 30 [Diesel]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
 
Node 30 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 32 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 33 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 34 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 35 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 36 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 37 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
Node 38 [Renewable]: real power = None MW | reactive power = None MVAr | apparent power = None MVA
 
